# Линейная регрессия

![](https://habrastorage.org/files/256/a5d/ed0/256a5ded03274e0f87ccf97164c31c35.png)

## Теория

Линейная регрессия - один из наиболее хорошо изученных методов машинного обучения, позволяющий прогнозировать значения количественного признака в виде линейной комбинации прочих признаков с параметрами - весами модели. Оптимальные (в смысле минимальности некоторого функционала ошибки) параметры линейной регрессии можно найти аналитически с помощью нормального уравнения или численно с помощью методов оптимизации.

Пусть есть некоторый числовой целевой признак $y$. Извеcтны признаки $X = [x_1, x_2, ... ,x_m]$. Модель регрессии предсказывает $y$ с помощью вычисления следующей взвешенной суммы:

$$
y = w_0 + \sum_{i=1}^{m} {w_i x_i} \tag{1}
$$

Добавив фиктивный признак $x_0 = 1$, можно будет записать:

$y = \sum_{i=0}^m w_i x_i = \vec{w}^T \vec{x} \tag{2}$

Если представить себе $X$ как матрицу объектов-признаков, $y$ - вектор столбец целевой переменной, а $w$ - вектор-столбец коэффициентов, то получим матричную запись:

$$
\Large \vec{y} = X\vec{w} \tag{3}
$$

## MSE — Среднеквадратичный Функция потерь

Очевидно, что в уравнениях $(1 - 3)$ нам известны значения $X$ и $y$ — это константы в некотором роде, так как это обучающая выборка. 

Чтобы подобрать значения коэффициентов (они же веса) $w$, нам необходима функция ошибки (loss-функция):

$$
MSE\left(\overline{y}, y\right) = \frac{1}{n} \sum_{i=0}^{n} {\left(y_i - \overline{y_i}\right)^2}, \tag{4}
\\
$$
$$
MSE(X, w_0, ... , w_1, y) = \frac{1}{n} \sum_{i=1}^{n} {\left(y_i - w_0 + w_1 x_1 + ... + w_n x_n\right)^2} \tag{5}
$$

Функция ошибки позволяет оценить точность модели в числовом виде — чем меньше ее значение, тем выше точность модели.

## МНК — Метод Наименьших Квадратов

Один из способов вычислить значения параметров модели является метод наименьших квадратов (МНК), который минимизирует среднеквадратичную ошибку между реальным значением зависимой переменной и прогнозом, выданным моделью:
$$
\large
\begin{array}
{rcl}\mathcal{L}\left(X, \vec{y}, \vec{w} \right) &=& \frac{1}{2n} \sum_{i=1}^n \left(y_i - \vec{w}^T \vec{x}_i\right)^2 \\
&=& \frac{1}{2n} \left\| \vec{y} - X \vec{w} \right\|_2^2 \\
&=& \frac{1}{2n} \left(\vec{y} - X \vec{w}\right)^T \left(\vec{y} - X \vec{w}\right)
\end{array}
$$

Для решения данной оптимизационной задачи необходимо вычислить производные по параметрам модели, приравнять их к нулю и решить полученные уравнения относительно $\vec w$ (частный случай был в лекции):

$$
\large
\begin{array}
{rcl} \frac{\partial \mathcal{L}}{\partial \vec{w}} = 0 &\Leftrightarrow& \frac{1}{2n} \left(-2 X^T \vec{y} + 2X^T X \vec{w}\right) = 0 \\
&\Leftrightarrow& -X^T \vec{y} + X^T X \vec{w} = 0 \\
&\Leftrightarrow& X^T X \vec{w} = X^T \vec{y} \\
&\Leftrightarrow& \vec{w} = \left(X^T X\right)^{-1} X^T \vec{y}
\end{array}
$$

### Нормальное уравнение

Таким образом для аналитического вычисления весов линейной регрессии нужно решить это нормальное уравнение: 

$$
\Large \vec{w} = \left(X^T X\right)^{-1} X^T \vec{y} \tag{6}
$$

Матрица ${(X^TX)}^{-1}X^T$ - [псевдообратная](https://ru.wikipedia.org/wiki/%D0%9F%D1%81%D0%B5%D0%B2%D0%B4%D0%BE%D0%BE%D0%B1%D1%80%D0%B0%D1%82%D0%BD%D0%B0%D1%8F_%D0%BC%D0%B0%D1%82%D1%80%D0%B8%D1%86%D0%B0) для матрицы $X$. В NumPy такую матрицу можно вычислить с помощью функции [numpy.linalg.pinv](http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.linalg.pinv.html).
Однако, нахождение псевдообратной матрицы - операция вычислительно сложная и нестабильная в случае малого определителя матрицы $X$ (проблема [мультиколлинеарности](https://ru.wikipedia.org/wiki/%D0%9C%D1%83%D0%BB%D1%8C%D1%82%D0%B8%D0%BA%D0%BE%D0%BB%D0%BB%D0%B8%D0%BD%D0%B5%D0%B0%D1%80%D0%BD%D0%BE%D1%81%D1%82%D1%8C)). На практике лучше находить вектор весов $w$ решением матричного уравнения $$\Large X^TXw = X^Ty$$Это может быть сделано с помощью функции [numpy.linalg.solve](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.linalg.solve.html).
Но все же на практике для больших матриц $X$ быстрее работает градиентный спуск, особенно его стохастическая версия.

## Градиентный спуск

### Градиент

Уравнение линейной регрессии завит от нескольких переменных (если число признаков больше одного), соответсвенно мы имеем дело с функцией нескольких переменных. Градиент такой функции — это вектор частных производных, указывающий направление наискорейшего _роста_ функции:
$$
\nabla{y(w_0, w_1, ... , w_n)} = \left({\frac{\partial y}{\partial w_0}}, {\frac{\partial y}{\partial w_1}}, ... , {\frac{\partial y}{\partial w_n}} \right)^T \tag{7}
$$

Посчитаем градиент для функции потерь MSE $(5)$:

$$
\large
\begin{array}
{rcl}\mathcal{L} &=& MSE(w_0, w_1, ... , w_n),\\
&=& \frac{1}{n} \sum_{i=1}^{n} {\left(y_i - w_0 + w_1 x_1 + ... + w_n x_n\right)^2}
\end{array}
$$

Частные производные по $w_0$ и $w_i$ отдельно:

$$
\large
\begin{align*}
\frac{\partial\mathcal{L}}{\partial w_0} =& \frac{2}{n} \sum_{i=1}^{n} {\left(w_0 + w_1 x_1 + ... + w_n x_n - y_i\right)}, \tag{8} \\
\frac{\partial\mathcal{L}}{\partial w_j} =& \frac{2}{n} \sum_{i=1}^{n} {x_j \left(w_0 + w_1 x_1 + ... + w_n x_n - y_i\right)},\ j \in \{1, \dots , n\} \tag{9}
\end{align*}
$$

Итого:

$$
\large
\frac{\partial\mathcal{L}}{\partial W_j} = \frac{2}{n} \sum_{i=1}^{n} {x_i^j \left(\overline{y}_i - y_i\right)} \tag{10}
$$


Параметры $w_0, w_1, w_2, w_3$, по которым минимизируется среднеквадратичная ошибка, можно находить численно с помощью градиентного спуска. Градиентный шаг для весов будет выглядеть следующим образом: 

$$
\begin{align*}
w_0 \leftarrow& w_0 - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}}, \\
w_j \leftarrow& w_j - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}},\ j \in \{1,2,3\}
\end{align*}
$$ 

Здесь $\eta$ - параметр, шаг градиентного спуска.

Тогда алгоритм на псевдокоде (из ШАДовского учебника по ML): 

```python
w = random_normal()              # можно пробовать и другие виды инициализации
repeat S times:                  # другой вариант: while abs(err) > tolerance
    f = X.dot(w)                 # посчитать предсказание
    err = f - y                  # посчитать ошибку
    grad = 2 * X.T.dot(err) / N  # посчитать градиент
    w -= alpha * grad            # обновить веса
```

### Стохастический градиентный спуск

Проблема градиентного спуска, описанного выше, в том, что на больших выборках считать на каждом шаге градиент по всем имеющимся данным может быть очень вычислительно сложно. В стохастическом варианте градиентного спуска поправки для весов вычисляются только с учетом одного случайно взятого объекта обучающей выборки: 

$$
\begin{align*}
w_0 \leftarrow& w_0 - \frac{2\eta}{\ell} {((w_0 + w_1x_{k1} + w_2x_{k2} +  w_3x_{k3}) - y_k)}, \\ 
w_j \leftarrow& w_j - \frac{2\eta}{\ell} {x_{kj}((w_0 + w_1x_{k1} + w_2x_{k2} +  w_3x_{k3}) - y_k)},\ j \in \{1,2,3\}
\end{align*}
$$

где $k$ - случайный индекс, $k \in \{1, \ldots, \ell\}$.

## Литература 

- [Учебник по ML от ШАД — линейные модели](https://ml-handbook.ru/chapters/linear_models/intro)
- [Первый конспект лекции про линейную регрессию из курса ФШЭ](https://github.com/esokolov/ml-course-hse/blob/master/2016-fall/lecture-notes/lecture02-linregr.pdf)
- [Второй конспект лекции про линейную регрессию из курса ФШЭ](https://github.com/esokolov/ml-course-hse/blob/master/2016-fall/lecture-notes/lecture03-linregr.pdf)
- [Теория из курса ODS](https://github.com/Yorko/mlcourse_open/blob/master/jupyter_notebooks/topic04_linear_models/topic4_linear_models_part1_mse_likelihood_bias_variance.ipynb)
- [Материалы из курса от МФТИ](https://www.coursera.org/specializations/machine-learning-data-analysis)
- [Статья про лин.рег. от ODS на habrahabr](https://habrahabr.ru/company/ods/blog/323890/)

В домашнем задании надо будет вычислить всё **вручную** методом нормального уравнения.

![](https://ebanoe.it/wp-content/uploads/2016/03/alkogolik-question-how-to-live.jpg)

## Практика

В этом заданиии мы будем использовать данные [SOCR](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_Dinov_020108_HeightsWeights) по росту и весу 25 тысяч подростков.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
METER_TO_INCH, KILO_TO_POUND = 39.37, 2.20462 # это константы конвертации метров в дюймы, килограмов в фунты

df = pd.read_csv('./data/weights_heights.csv.gz', index_col='Index')
df['Height'] = df['Height'] / METER_TO_INCH
df['Weight'] = df['Weight'] / KILO_TO_POUND
df.head()

### Визуализируем имеющиеся данные



In [ ]:
# Гистограмма значений роста
df.plot(y='Height', kind='hist', 
        color='red',  title='Height (m.) distribution')

In [ ]:
# Гистограмма значений веса
df.plot(y='Weight', kind='hist', 
        color='green',  title='Weight (kg.) distribution')

In [ ]:
# посмотрим на scatter плот относительно роста и веса
# Видно что есть некоторая линейная зависимость между этими параметрами
df.plot(kind='scatter', y='Weight', x='Height', title='Height vs Weight')

Что можно сказать про эти данные по их графикам?

### Среднеквадратичная ошибка

В простейшей постановке задача прогноза значения вещественного признака по прочим признакам (задача восстановления регрессии) решается минимизацией квадратичной функции ошибки.

В нашем случае у нас 2 признака поэтому будет два параметра $w_0$ и $w_1$

$$error(w_0, w_1) = \sum_{i=1}^n {(y_i - (w_0 + w_1 * x_i))}^2 $$

In [ ]:
def error(x1, y, w0, w1):
    ### Ваш код ###

Зафиксируем какие-нибудь значения `w0` и `w1`.  
Посмотрим как изменяется значение ошибки `error` в зависимости от `w0`,`w1`.

Если всё верно, то в нижней ячейке должны быть выведены числа `209.69670759713378` и `226.27746779896154`, с точностью до 8-10 знака после запятой

In [ ]:
weight, height = df['Weight'].values, df['Height'].values
print(error(weight,height, 1.3, 0.006))
print(error(weight,height, 0.6, 0.02))

Нарисуем на scatter plot эти две прямые, чтобы посмотреть как они "улавливают" закономерность.  
Очевидно по w0, w1 можно задать уравнение прямой.

In [ ]:
plt.figure(figsize=(12,8))
# alpha - это прозрачность точек, edgecolors - это цвет края точки
plt.scatter(x=weight, y=height, edgecolors='black', alpha=0.5)
plt.title('Height vs. weight')
plt.xlabel('Weight')
plt.ylabel('Height')
x = np.linspace(start=30.0, stop=80.0, num=100)
plt.plot(x, 1.3 + 0.006 * x, c='green', linewidth=5)
plt.plot(x, 0.6 + 0.02 * x, c='red', linewidth=5)

Минимизация квадратичной функции ошибки - относительная простая задача, поскольку функция выпуклая. Для такой задачи существует много методов оптимизации. Посмотрим, как функция ошибки зависит от одного параметра (наклон прямой), если второй параметр (свободный член) зафиксировать.

In [ ]:
w0 = 1.6 # фиксируем свободный член
w1 = np.linspace(start=-0.02, stop=0.02, num=100) # перебираем значения w1

err = []
for w1_i in w1:
    err.append(error(weight, height, w0, w1_i))

plt.plot(w1, err)
plt.xlabel('w1')
plt.ylabel('error')
plt.title('error vs. w1 for w0=50')

Найдем оптимальное значение для $w_1$ при фиксированном $w_0 = 1.6$. Для этого воспользуемся методом `minimize_scalar`.

In [ ]:
from scipy.optimize import minimize_scalar
w0 = 1.6
w1_opt = minimize_scalar(
    fun=lambda x: error(weight, height, w0, x), 
    bounds=(-0.2, 0.2)).x
w1_opt

In [ ]:
plt.figure(figsize=(12,8))

plt.scatter(x=weight, y=height, edgecolors='black', alpha=0.5)
plt.title('Height vs. weight')
plt.xlabel('Weight')
plt.ylabel('Height')

x = np.linspace(start=30.0, stop=80.0, num=100)
plt.plot(x, w0 + w1_opt * x, c='green', linewidth=5)

В прошлых графиках мы фиксировали $w_0$ и рисовали двумерные график ошибок. Но по сути мы должны перебирать и $w_0$ и $w_1$, а это трёхмерный график. Его тоже можно визулизировать. (Если вам нужны интерактивные графики, то посмотрите библиотеку [plotly](https://plotly.com/python)).

In [ ]:
# функция работает со всем датасетом и с мешгридом w0/w1
# чисто для учебных целей
def error(x, y, w0, w1):
    return sum(map(lambda xi, yi: ((yi - (w0 + w1 * xi))**2), x, y))

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(14,10))
ax = fig.gca(projection='3d') # get current axis

w0 = np.linspace(start=-3, stop=3, num=100)
w1 = np.linspace(start=-2, stop=2, num=100)
w0, w1 = np.meshgrid(w0, w1)
z = error(weight, height, w0, w1)

ax.plot_surface(w0, w1, z)
ax.set_xlabel('Intercept')
ax.set_ylabel('Slope')
ax.set_zlabel('Error')
pass

Теперь попробуем реализовать то же самое, только с помощью метода наименьших квадратов и для более общего случая.

Теория на этот счёт изложена в начале ноутбука.

Вспомним самое важное для нас.

Нормальное уравнение в матричной форме:

$$\Large X^TXw = X^Ty$$

Искомая часть для нас здесь $-$ переменная $w$. Это матрица, т.к. уравнение матричное.

Вычислить её можно с помощью функции [numpy.linalg.solve](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.linalg.solve.html)

### Дополните реализацию линейной модели

In [ ]:
class LeastSquaresRegression:
    # конструктор класса
    def __init__(self):
        # для ясности заранее объявим поле, по умолчанию оно пустое, но потом будет заполнено вычисленными весами
        self.w = None
    # магия, которая превращает наш объект в callable
    def __call__(self, x, *args, **kwargs):
        return self.predict(x)

    # этот метод вычисляет веса модели и записывает их в переменную w у текущего объекта
    def fit(self, x, y):
        #############
        #  Ваш код  #
        #############

    # этот метод производит вычисление y из уравнения выше (советую сначала вывести на бумаге)
    def predict(self, x):
        #############
        #  Ваш код  #
        #############

Модель получилась максимально простой, поэтому для эмуляции свободного члена ($w_0$) нам понадобится модифицировать данные.

Раз уж мы решили, что рост откладывается на Oy, то давайте считать, что вес это Х.

**Создайте таблицу Х и задайте у неё два столбца - одно это вес из наших исходных данных, а второе это столбец полностью из единичек**

Убедитесь, что столбец с весом идёт первым.

In [ ]:
X = pd.DataFrame(columns=['Weight', 'Intercept'])
X['Weight'] = weight
X['Intercept'] = np.ones(shape=X.shape[0])
X.head()

Теперь обучите Вашу самодельную модель и выведите её веса

In [ ]:
clf = LeastSquaresRegression()

clf.fit(X, height)

print(clf.w)

Дальше идёт стадия проверки

Найдем минимум функции ошибки по трехмерной функции с помощью метода `minimize` и оптимизатора `L-BFGS-B`.

In [ ]:
from scipy.optimize import minimize

(w0_opt, w1_opt) = minimize(
    fun=lambda x: error(weight, height, x[0], x[1]), 
    x0=(0, 0), 
    bounds=((-3, 3), (-2, 2)), 
    method='L-BFGS-B').x
print(f"w0_opt={w0_opt}\nw1_opt={w1_opt}")

Точки в нашем классификаторе и в ячейке сверху должны быть одинаковые. Иначе ошибка

In [ ]:
plt.figure(figsize=(12,8))

plt.scatter(x=weight, y=height, edgecolors='black', alpha=0.5)
plt.title('Height vs. weight')
plt.xlabel('Weight')
plt.ylabel('Height')

x = np.linspace(start=30.0, stop=80.0, num=100)
plt.plot(x, w0_opt + w1_opt * x, c='green', linewidth=5)
plt.plot(x, clf.w[1] + clf.w[0] * x, c='yellow', linewidth=2)

Если всё реализовано верно, то зелёная и жёлтая линии должны быть друг на друге

## Регуляризация Lasso и Ridge

Рассмотрим, как переобучаются линейные модели, почему так происходит, и выясним, как диагностировать и контролировать переобучение с помощью регуляризации.

Возьмем известный нам уже датасет о прокатах велосипедов. Его можно рассматривать в свете регрессии и предсказывать количество прокатов в зависимости от погоды.

```
season: 1 - весна, 2 - лето, 3 - осень, 4 - зима
yr: 0 - 2011, 1 - 2012
mnth: от 1 до 12
holiday: 0 - нет праздника, 1 - есть праздник
weekday: от 0 до 6
workingday: 0 - нерабочий день, 1 - рабочий день
weathersit: оценка благоприятности погоды от 1 (чистый, ясный день) до 4 (ливень, туман)
temp: температура в Цельсиях
atemp: температура по ощущениям в Цельсиях
hum: влажность
windspeed(mph): скорость ветра в милях в час
windspeed(ms): скорость ветра в метрах в секунду
cnt: количество арендованных велосипедов (это целевой признак, его мы будем предсказывать)
```

In [ ]:
df = pd.read_csv('./data/bikes_rent.csv.gz') 
df.head()

In [ ]:
df.describe()

Построим матрицу корреляций признаков, чтобы найти линейно зависимые признаки.  
Обратите внимание на то, что мы берем абсолютное значение корреляции. Нам не важно как коррелируют - положительно или отрицательно. Главное, что чем больше корреляция или антикорреляция, тем сильнее линейная связь между признаками.

In [ ]:
# корреляционная матрица
corr = np.abs(df.corr())

In [ ]:
# эта маска нужна для того, чтобы красиво вывести heatmap, без дублирования данных
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

In [ ]:
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr,cmap=cmap, square=True,linewidths=.5, mask=mask,annot=True, fmt=".1f")

Судя по корреляционной матрице есть несколько признаков который сильно коррелируют с целевой переменной - числом прокатов (`cnt`):

- yr
- temp
- atemp
- season

Как мы помними в линейной регрессии используются взвешенные суммы признаков, поэтому все они должны быть одного масштаба. (Что бы избежать расхождения алгоритма градиентного спуска) Проверим, так ли это у нас.

In [ ]:
df.mean()

Как мы видим все они разного масштаба, следовательно их требуется отмасштабировать. Можно использовать простой `sklearn.preprocessing.StandardScaler`. Суть его очень простая - приводить всё к нулевому среднему и масштабировать до единичного значения разброса. Получается такая формула для вычисления значения признака: 
$$x_i = \frac{x_i - mean(X)}{stdev(X)}$$

Тогда $Mean(X) = 0$ и $Stdev(X) = 1$

In [ ]:
# Посмотим еще раз на изначальные распределения признаков

sns.kdeplot(df['temp'])
sns.kdeplot(df['atemp'])
sns.kdeplot(df['windspeed(ms)'])
sns.kdeplot(df['windspeed(mph)'])
sns.kdeplot(df['hum'])

In [ ]:
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [ ]:
df = shuffle(df, random_state=42)
X = df.drop(['cnt'], axis=1)
y = df['cnt']

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X.mean(axis=0)

In [ ]:
X.std(axis=0)

Как видим стандартное отклонение теперь единично, а среднее близко к нулю.

In [ ]:
plt.figure(figsize=(12,8))
for i in range(X.shape[1]):
    sns.kdeplot(X[i])

Теперь обучим линейную модель. В этот раз мы будем делать это не вручную, а использовать класс `LinearRegression`.

Воспользуемся им для начала без параметров, то есть со значениями по умолчанию.

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X, y)  # обучение. Может выдать warning про LAPACK - всё ок
for coef, col in sorted(zip(lr.coef_, df.columns), key=lambda x: np.abs(x[0])):
    print("{} \t {}".format(np.round(coef, 4), col))

Посмотрим на магнитуду признаков, то есть на зхначения весов в линейной модели при каждом признаке. Мы помним, что чем , больше по модулю коэффициент, тем больше "веса" он имеет в модели и тем важнее признак.

In [ ]:
# Магнитуда признаков
plt.figure(figsize=(14,8))
sns.barplot(y='columns', x='coef',data=pd.DataFrame(list(zip(lr.coef_, df.columns)), columns=["coef", "columns"]))

Мы видим, что веса при линейно-зависимых признаках по модулю значительно больше, чем при других признаках.
Чтобы понять, почему так произошло, вспомним аналитическую формулу, по которой вычисляются веса линейной модели в методе наименьших квадратов: $w = (X^TX)^{-1} X^T y$.

Если в $X$ есть коллинеарные (линейно-зависимые) столбцы, матрица $X^TX$ становится вырожденной, и формула перестает быть корректной. Чем более зависимы признаки, тем меньше определитель этой матрицы и тем хуже аппроксимация $Xw \approx y$. Такая ситуацию называют проблемой мультиколлинеарности, мы обсуждали ее на лекции.

С парой `temp`-`atemp` чуть менее коррелирующих переменных такого не произошло, однако на практике всегда стоит внимательно следить за коэффициентами при похожих признаках.

Решение проблемы мультиколлинеарности состоит в регуляризации линейной модели. К оптимизируемому функционалу прибавляют $L_1$ или $L_2$ норму весов, умноженную на коэффициент регуляризации $\alpha$. В первом случае метод называется **Lasso**, а во втором – **Ridge**.

#### Lasso (L1)

$$\Large error(X, y, w) = \frac{1}{2} \sum_{i=1}^\ell {(y_i - w^Tx_i)}^2 + \alpha \sum_{i=1}^d |w_i|$$, где $\alpha$ - это коэффициент регуляризации.

#### Ridge (L2)

$$\Large error(X, y, w) = \frac{1}{2} \sum_{i=1}^\ell {(y_i - w^Tx_i)}^2 + \alpha \sum_{i=1}^d w_i^2$$, где $\alpha$ - это коэффициент регуляризации.

Для Ridge (Так же известного как регуляризация Тихонова) можно переформулировать матричное уравнение в нормальной форме:
$$\Large (X^TX - \lambda I)w = X^Ty$$
А для Lasso нужен координатный спуск. Про концепцию алгоритма вы можете почитать [здесь](http://www.machinelearning.ru/wiki/index.php?title=%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%BF%D0%BE%D0%BA%D0%BE%D0%BE%D1%80%D0%B4%D0%B8%D0%BD%D0%B0%D1%82%D0%BD%D0%BE%D0%B3%D0%BE_%D1%81%D0%BF%D1%83%D1%81%D0%BA%D0%B0).

---

Рассмотрим на примере как разная регуляризация влияет на веса $w$ модели.

Попробуем реализовать Ridge руками

In [ ]:
class Ridge(LeastSquaresRegression):
    def __init__(self, alpha=0.1):
        super(Ridge, self).__init__()
        self.alpha = alpha

    def fit(self, x, y):
        #############
        #  Ваш код  #
        #############

In [ ]:
ridge = Ridge()
ridge.fit(X, y)

for coef, col in sorted(zip(ridge.w, df.columns), key=lambda x: np.abs(x[0])):
    print("{} \t {}".format(np.round(coef, 4), col))
    
# Магнитуда признаков
plt.figure(figsize=(14,8))
sns.barplot(y='columns', x='coef',data=pd.DataFrame(list(zip(ridge.w, df.columns)), columns=["coef", "columns"]))
pass

Теперь сравните наш Ridge с библиотечным:

In [ ]:
from sklearn.linear_model import Lasso, Ridge
ridge = Ridge()
ridge.fit(X, y)

for coef, col in sorted(zip(ridge.coef_, df.columns), key=lambda x: np.abs(x[0])):
    print("{} \t {}".format(np.round(coef, 4), col))
    
# Магнитуда признаков
plt.figure(figsize=(14,8))
sns.barplot(y='columns', x='coef',data=pd.DataFrame(list(zip(ridge.coef_, df.columns)), columns=["coef", "columns"]))
pass

In [ ]:
lasso = Lasso()
lasso.fit(X, y)

for coef, col in sorted(zip(lasso.coef_, df.columns), key=lambda x: np.abs(x[0])):
    print("{} \t {}".format(np.round(coef, 4), col))
    
plt.figure(figsize=(14,8))
sns.barplot(y='columns', x='coef',data=pd.DataFrame(list(zip(lasso.coef_, df.columns)), columns=["coef", "columns"]))
pass

Как мы видим, Lasso контролирует веса модели **обнуляя веса** при бесполезных признаках.  
Ridge просто **контролирует величину** коэффициентов. 


В любом случае регуляризатор просто вводит штрафы на очень большие веса $w$.

#### Коэффициент регуляризации

Посмотрим как изменение коэффициента регуляризации влияет на веса.

In [ ]:
alphas = np.arange(1, 500, 5)
coefs_lasso = np.zeros((alphas.shape[0], X.shape[1])) # матрица весов размера (число регрессоров) x (число признаков)
coefs_ridge = np.zeros((alphas.shape[0], X.shape[1]))

i = 0
for alpha in alphas:
    lasso = Lasso(alpha = alpha, random_state=42)
    lasso.fit(X, y)
    coefs_lasso[i, :] = lasso.coef_
    
    ridge = Ridge(alpha = alpha, random_state=42)
    ridge.fit(X, y)
    coefs_ridge[i, :] = ridge.coef_
    
    i += 1

In [ ]:
plt.figure(figsize=(10, 8))
for coef, feature in zip(coefs_lasso.T, df.columns):
    plt.plot(alphas, coef, label=feature, color=np.random.rand(3), linewidth=3)
plt.legend(loc="upper right", bbox_to_anchor=(1.4, 0.95))
plt.xlabel("alpha")
plt.ylabel("feature weight")
plt.title("Lasso")

plt.figure(figsize=(10, 8))
for coef, feature in zip(coefs_ridge.T, df.columns):
    plt.plot(alphas, coef, label=feature, color=np.random.rand(3), linewidth=3)
plt.legend(loc="upper right", bbox_to_anchor=(1.4, 0.95))
plt.xlabel("alpha")
plt.ylabel("feature weight")
plt.title("Ridge")

- Судя по графику Lasso($L_1$) агрессивнее уменьшает веса, чем Ridge($L_2$)
- В случае с $L_1$ регуляризатором, увеличение alpha сколлапсирует все веса в ноль. Это происходит из-за характера функции регуляризации. В $L_1$ - это $sum(|W|) * alpha$ в то время как в $L_2$ это $sum(W^2) * alpha$. Тогда $L_1$ слагамемое регуляризации будет больше чем слагаемое $L_2$ регуляризации. А оптимальное значение будет достигаться в 0. Кроме того $L_1$ производит отбор признаков, зануляя неинформативные.

### Выбор лучшего alpha

Итак, мы видим, что при изменении alpha модель по-разному подбирает коэффициенты признаков. Нам нужно выбрать наилучшее alpha.

Для этого, во-первых, нам нужна метрика качества. Будем использовать в качестве метрики сам оптимизируемый функционал метода наименьших квадратов, то есть _Mean Square Error_.

Во-вторых, нужно понять, на каких данных эту метрику считать. Нельзя выбирать alpha по значению MSE на обучающей выборке, потому что тогда мы не сможем оценить, как модель будет делать предсказания на новых для нее данных. Если мы выберем одно разбиение выборки на обучающую и тестовую (это называется holdout), то настроимся на конкретные "новые" данные, и вновь можем переобучиться. Поэтому будем делать несколько разбиений выборки, на каждом пробовать разные значения alpha, а затем усреднять MSE. Удобнее всего делать такие разбиения кросс-валидацией, то есть разделить выборку на K частей, или блоков, и каждый раз брать одну из них как тестовую, а из оставшихся блоков составлять обучающую выборку.

Делать кросс-валидацию для регрессии в sklearn совсем просто: для этого есть специальный регрессор, LassoCV, который берет на вход список из alpha и для каждого из них вычисляет MSE на кросс-валидации. После обучения (если оставить параметр `cv=3`) регрессор будет содержать переменную `mse_path_`, матрицу размера `len(alpha)` x `k`, `k = 3` (число блоков в кросс-валидации), содержащую значения MSE на тесте для соответствующих запусков. Кроме того, в переменной `alpha_` будет храниться выбранное значение параметра регуляризации, а в `coef_`, традиционно, обученные веса, соответствующие этому `alpha_`.

In [ ]:
from sklearn.linear_model import LassoCV

alphas = np.linspace(0.001, 50, 500)
lasso_cv = LassoCV(alphas=alphas, random_state=42)
lasso_cv.fit(X, y)

mean_mse = np.mean(lasso_cv.mse_path_, axis = 1)

plt.plot(lasso_cv.alphas_, mean_mse)
plt.xlabel('alpha')
plt.ylabel('MSE')
plt.title('Lasso')
print('alpha = {}'.format(lasso_cv.alpha_))

Посмотрим на значения коэффициентов при оптимальном alpha.

In [ ]:
coef = pd.DataFrame(list(zip(np.round(lasso_cv.coef_, 2), df.columns)))
coef.columns = ['weight', 'feature']
coef.sort_values(['weight'], ascending=False)

После построения модели, подбора параметров и обучения наступает этап анализа результатов.  
На сколько хорошо модель строит предсказания, можно ли ей пользоваться, где она ошибается чаще всего? 

Для того, чтобы ответить на эти вопросы, мы сделаем следующее:

1. Проверим с помощью кросс валидации значение средней ошибки. Кросс валидация - это случайное разбиение выборки на части, обучение модели и вычисление средней точности модели.
2. Оценим долю ошибок
3. Визуализируем ошибки

### Кросс валидация

> Этот блок взят из курса ODS: https://habr.com/company/ods/blog/322534/

Главная задача обучаемых алгоритмов – их способность обобщаться, то есть хорошо работать на новых данных. Поскольку на новых данных мы сразу не можем проверить качество построенной модели (нам ведь надо для них сделать прогноз, то есть истинных значений целевого признака мы для них не знаем), то надо пожертвовать небольшой порцией данных, чтоб на ней проверить качество модели.


Чаще всего это делается одним из 2 способов:

- отложенная выборка (held-out/hold-out set). При таком подходе мы оставляем какую-то долю обучающей выборки (как правило от 20% до 40%), обучаем модель на остальных данных (60-80% исходной выборки) и считаем некоторую метрику качества модели (например, самое простое – долю правильных ответов в задаче классификации) на отложенной выборке.
- кросс-валидация (cross-validation, на русский еще переводят как скользящий или перекрестный контроль). Тут самый частый случай – K-fold кросс-валидация.

![image.png](https://habrastorage.org/files/b1d/706/e6c/b1d706e6c9df49c297b6152878a2d03f.png)

Тут модель обучается  раз на разных () подвыборках исходной выборки (белый цвет), а проверяется на одной подвыборке (каждый раз на разной, оранжевый цвет).
Получаются  оценок качества модели, которые обычно усредняются, выдавая среднюю оценку качества классификации/регрессии на кросс-валидации.


Кросс-валидация дает лучшую по сравнению с отложенной выборкой оценку качества модели на новых данных. Но кросс-валидация вычислительно дорогостоящая, если данных много.


Кросс-валидация – очень важная техника в машинном обучении (применяемая также в статистике и эконометрике), с ее помощью выбираются гиперпараметры моделей, сравниваются модели между собой, оценивается полезность новых признаков в задаче и т.д. Более подробно можно почитать, например, [тут у Sebastian Raschka](https://sebastianraschka.com/blog/2016/model-evaluation-selection-part1.html) или в любом классическом учебнике по машинному (статистическому) обучению.

In [ ]:
from sklearn.model_selection import cross_val_score
cv = 3 # проведем 3 эксперимента
errors = cross_val_score(Lasso(lasso_cv.alpha_), X, y, cv=cv, scoring='neg_mean_absolute_error')
print(errors)
error = abs(np.mean(errors))
print("error={:0.3f}, std={:0.4f}".format(error, np.std(errors)))

# судя по отклонению модель получилась не очень стабильной

Но что вообще означает эта ошибка?

In [ ]:
total_cnt = df.cnt.sum()
print("Прокатов было совершено за всё время", total_cnt)

In [ ]:
"Доля ошибок: {:.2%}".format((error*X.shape[0]) / total_cnt)

### Визуализация ошибки регрессии

Интересно же посмотреть на графике где и как ошибается предсказание!

Для начала разобьем всю выборку на test и train. То есть на train сегменте мы будем обучать модель, а на test - проверять качество.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print("train_shape={}, test_shape={}".format(X_train.shape, X_test.shape))

reg = Lasso(lasso_cv.alpha_)
reg.fit(X_train, y_train)

pred = reg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

# Построим таблицу предсказание / оригинальное значение / MSE / MAE
prediction = pd.DataFrame(np.vstack((pred, y_test)).T, columns=["pred", "target"])
prediction['MSE'] = np.power(prediction['pred'] - prediction['target'], 2)
prediction['MAE'] = np.abs(prediction['pred'] - prediction['target'])

In [ ]:
# Сортируем по убыванию ошибки
prediction.sort_values(by='MAE', ascending=False, inplace=True)
prediction.head()

In [ ]:
# Сортируем по возрастанию ошибки
prediction.sort_values(by='MAE', inplace=True)
prediction.head()

In [ ]:
print("общая ошибка предсказателя в количестве прокатов = {:0.1f}\nвсего прокатов = {:0.1f}\nдоля ошибок = {:.2%}"
      .format(prediction.MAE.sum(), prediction.target.sum(), prediction.MAE.sum() / prediction.target.sum()))

In [ ]:
plt.plot(range(0, prediction.shape[0]), prediction['MAE'])
plt.xlabel("Index")
plt.ylabel("MAE")

Финальная визуализация:  
На одном графике target и prediction. Точки отсортированы по величине ошибки.

In [ ]:
plt.figure(figsize=(14,7))

plt.plot(range(0, prediction.shape[0]), prediction['pred'])
plt.plot(range(0, prediction.shape[0]), prediction['target'])
plt.plot(range(0, prediction.shape[0]), prediction['MAE'])
plt.legend(["Predicted", "Target", "MAE"])

plt.xlabel("Element index")
plt.ylabel("Target/Predicted value")
pass

Вывод делайте сами, достаточно ли погрешности в 15% от всего проката или нет :)

## Бонус секция

Пример с полиномиальной регрессией и проблемой мультиколлинеарности, как было рассказано на лекции.

Взято из материалов мастеркласса Aiarlabs https://github.com/mephistopheies/mlworkshop39_042017.

In [ ]:
def generate_wave_set(n_support=1000, n_train=25, std=0.3):
    data = {}
    # create 1000 points uniformly distibuted in closed intervl from 0 to 2*pi
    # it is some kind of resolution for sampling
    data['support'] = np.linspace(0, 2*np.pi, num=n_support)
    # calculate sine values
    data['values'] = np.sin(data['support']) + 1
    # choose n_train random vakues from support (with replacement) and sort them ascending
    data['x_train'] = np.sort(np.random.choice(data['support'], size=n_train, replace=True))
    # calculate sine for each of samplesd point
    data['y_train'] = np.sin(data['x_train']) + 1 + np.random.normal(0, std, size=data['x_train'].shape[0])
    return data

In [ ]:
# sample 25 points from 1000 available from noised sin manifold 
data = generate_wave_set(1000, 25)

print('Shape of X is', data['x_train'].shape)
print('Head of X is', data['x_train'][:10])

margin = 0.3
plt.figure(figsize=(12, 8))
plt.plot(data['support'], data['values'], 'b--', alpha=0.5, label='manifold')
plt.scatter(data['x_train'], data['y_train'], 40, 'g', 'o', alpha=0.8, label='data')
plt.xlim(data['x_train'].min() - margin, data['x_train'].max() + margin)
plt.ylim(data['y_train'].min() - margin, data['y_train'].max() + margin)
plt.legend(loc='upper right', prop={'size': 20})
plt.title('True manifold and noised data', fontsize=20)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
# define list with degrees of polynomials to investigate
degree_list = [1, 2, 3, 10, 12, 13]
# get color palette
cmap = plt.get_cmap('jet')
# compute individual color for each curve
colors = [cmap(i) for i in np.linspace(0, 1, len(degree_list))]

margin = 0.3
plt.figure(figsize=(16, 12))
plt.plot(data['support'], data['values'], 'b--', alpha=0.5, label='manifold')
plt.scatter(data['x_train'], data['y_train'], 40, 'g', 'o', alpha=0.8, label='data')

# save weights of all fitted polynomial regressions
w_list = []
err = []
for ix, degree in enumerate(degree_list):
    # list with polynomial features for each degree
    dlist = [np.ones(data['x_train'].shape[0])] + list(map(lambda n: data['x_train']**n, range(1, degree + 1)))
    X = np.array(dlist).T
    w = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), data['y_train'])
    w_list.append((degree, w))
    y_hat = np.dot(w, X.T)
    err.append(np.mean((data['y_train'] - y_hat)**2))
    plt.plot(data['x_train'], y_hat, color=colors[ix], label='poly degree: %i' % degree)

plt.xlim(data['x_train'].min() - margin, data['x_train'].max() + margin)
plt.ylim(data['y_train'].min() - margin, data['y_train'].max() + margin)
plt.legend(loc='upper right', prop={'size': 20})
plt.title('Fitted polynomial regressions', fontsize=20)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

# Градиентный спуск в лин регрессии

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

x = np.array([[x] for x in np.arange(0.0, 6.0, 0.5)])
y = np.array([[2*x[0] + 10] for x in x.tolist()])

plt.scatter(x, y)
plt.show()

Сейчас у нас строго *линейная зависимость*. Добавим немного шума, используя метод `np.random.rand`

Воспользуемся `np.random.seed(someNumber)`, чтобы зафиксировать случайность и получать при перезапуске ячейки
стабильный результат.

**Hint**
- [Доки по np.random](https://docs.scipy.org/doc/numpy-1.15.0/reference/routines.random.html)

In [ ]:
np.random.seed(42)

x = np.array([[x] for x in np.arange(0.0, 6.0, 0.5)])
y = np.array([[2*x[0] + 10 + np.random.rand()*3] for x in x.tolist()])
plt.scatter(x, y)
plt.show()

In [ ]:
# Объявим параметры: веса и шаг градиентного спуска

w0 = np.random.rand()
w1 = np.random.rand()
step = 0.008

# Как видно, случайно инициализированные веса не очень хорошо приблизили линейную зависимость.
# Точнее вообще никак не приблизили
plt.scatter(x, y)
plt.plot(x, x*w1 + w0, c='red')

In [ ]:
# Реализуем само обучение
# Число шагов для обучения установим в 30

for i in range(30):
    
    # Предсказание
    pred = x*w1 + w0
    
    # MSE-loss
    loss = np.sum((y - pred)**2)/x.shape[0]
    
    # шаг для каждого из весов
    w0 = w0 - step*np.sum((pred - y) * x) * 2 / x.shape[0]
    w1 = w1 - step*np.sum(pred - y) * 2 / x.shape[0]
    
    # Критерий остановки обучения:
    # разница между значениями MSE меньше, чем на 0.01
    # Вычтем из старого значения лосс-функции новое, полученное уже после обновления весов. 
    if np.abs(loss - np.sum((x*w1 + w0 - y)**2) / x.shape[0]) < 0.01:
        print(loss)
        break
        

# Теперь красная линия приближает данные лучше.        
plt.scatter(x, y)
plt.plot(x, x*w1 + w0, c='red')

w1, w0

### Задание 

Реализовать модели линейной регрессии в виде класса, способного работать как с одномерным набором признаков(как в примере выше), так и с многомерным (когда у нас есть x_1, x_2 и тд).  
Веса желательно задать в виде вектора, X преобразовать, добавив в начало единицу (см. теорию).  
Используйте возможности библиотеки numpy (например, при использовании матриц, пользуйтесь [np.dot](https://numpy.org/doc/stable/reference/generated/numpy.dot.html), [np.transpose](https://numpy.org/doc/stable/reference/generated/numpy.transpose.html)).

**Учтите**, что признаки - это `numpy`-массивы:  таблицы с данными легко превратить в numpy-массив (это вы увидите позже), а сами по себе такие массивы удобнее, чем стандартные питоновские списки, в первую очередь за счет того, что 
- numpy быстрее, чем питон, так как под капотом написан на C,
- в библиотеке реализовано множество полезных методов, в чем вы уже могли убедиться.

In [ ]:
class LinearRegressionGrad:
    def __init__(self, lr = 0.0001, iters = 5000) -> None:
        self.lr = lr
        self.iters = iters

    def fit(self, x: np.ndarray, y: np.ndarray) -> None:
        # TODO: напиши меня!
    
    def predict(self, x: np.ndarray) -> np.ndarray:
        # TODO: напиши меня!
    
    def _getBiased(self, x):
        # это подсказка, помните мы добавляли фейковую колонку с 1 в датасет

Далее проверки работы вашего алгоритма

In [ ]:
from sklearn.datasets import make_regression
from matplotlib import pyplot as plt
np.random.seed(42)

lr = LinearRegressionGrad(iters=3000, lr=0.001)
lr.fit(x, y.reshape(-1))
pred = lr.predict(x)

plt.scatter(x, y)
plt.plot(x, pred, c='red')

lr.w

In [ ]:
# Проверим размерности
assert pred.shape == np.array(y.reshape(12)).shape

### Проверка на реальном датасете

Загрузим датасет с [прокатами велосипедов](https://www.kaggle.com/c/bike-sharing-demand/data).

Попробуем решить задачу регрессии, и предсказать число прокатов в зависимости от погоды.  
Описание всех признаков:  

```
season: 1 - весна, 2 - лето, 3 - осень, 4 - зима
yr: 0 - 2011, 1 - 2012
mnth: от 1 до 12
holiday: 0 - нет праздника, 1 - есть праздник
weekday: от 0 до 6
workingday: 0 - нерабочий день, 1 - рабочий день
weathersit: оценка благоприятности погоды от 1 (чистый, ясный день) до 4 (ливень, туман)
temp: температура в Цельсиях
atemp: температура по ощущениям в Цельсиях
hum: влажность
windspeed(mph): скорость ветра в милях в час
windspeed(ms): скорость ветра в метрах в секунду
cnt: количество арендованных велосипедов (это целевой признак, его мы будем предсказывать)
```

Предсказывать мы будем `cnt`, все остальные значения - нецелевые.

In [ ]:
df = pd.read_csv('./data/bikes_rent.csv.gz') 
df.head()

In [ ]:
import seaborn as sns

sns.kdeplot(df['temp'])
sns.kdeplot(df['atemp'])
sns.kdeplot(df['windspeed(ms)'])
sns.kdeplot(df['windspeed(mph)'])
sns.kdeplot(df['hum'])

In [ ]:
def standardScaler(x):
    mean = x - np.mean(x)
    return mean / np.std(x)

In [ ]:
# разобьем выборку на целевые и нецелевые признаки
# удалим windspeed(ms) сразу, так как мы уже знаем, что это плохой признак
x = df.drop(['cnt', 'windspeed(ms)'], axis=1)
y = df['cnt']

In [ ]:
x_scaled = standardScaler(x)
x_scaled

In [ ]:
plt.figure(figsize=(12,8))
for i in x_scaled.columns:
    sns.kdeplot(x_scaled[i])

Среднее должно быть околонулевым значением, а отклонение равно единице.

In [ ]:
print(np.mean(x_scaled.values))
print(np.std(x_scaled.values))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, random_state=42)

In [ ]:
model = LinearRegressionGrad(iters=5000, lr=0.001)
model.fit(X_train, y_train)
pred = model.predict(X_test)

mae = np.abs(pred - y_test).mean()
print(mae)

In [ ]:
model.w

In [ ]:
ridge = Ridge()
ridge.fit(X_train, y_train)
pred = ridge.predict(X_test)

mae = np.abs(pred - y_test).mean()
print(mae)

In [ ]:
ridge.coef_, ridge.intercept_